In [1]:
sc.master

'local[*]'

In [2]:
sc.stop()

In [3]:
import pyspark.sql.types 
from pyspark import SparkConf, SparkContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import  StringIndexer, OneHotEncoder,VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.sql.functions import udf,col
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql import SQLContext

global Path    
Path="file:/home/vivid/Downloads/"

In [4]:
def CreateSparkContext():
    def SetLogger( sc ):
        logger = sc._jvm.org.apache.log4j
        logger.LogManager.getLogger("org"). setLevel( logger.Level.ERROR )
        logger.LogManager.getLogger("akka").setLevel( logger.Level.ERROR )
        logger.LogManager.getRootLogger().setLevel(logger.Level.ERROR)    

    sparkConf = SparkConf().setAppName("RunDecisionTreeBinary").set("spark.ui.showConsoleProgress", "false") 
    sc = SparkContext(conf = sparkConf)
    print(("master="+sc.master))    
    SetLogger(sc)
    return (sc)

In [5]:
sc = CreateSparkContext()

master=local[*]


In [6]:
print("read data")
sqlContext = SQLContext(sc)

read data


In [9]:
row_df = sqlContext.read.format("csv").option("header", "true").load(Path+"data/hour.csv")

In [11]:
row_df

DataFrame[instant: string, dteday: string, season: string, yr: string, mnth: string, hr: string, holiday: string, weekday: string, workingday: string, weathersit: string, temp: string, atemp: string, hum: string, windspeed: string, casual: string, registered: string, cnt: string]

In [12]:
print(row_df.columns)

['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered', 'cnt']


In [15]:
df= row_df.drop("instant").drop("dteday") \
                            .drop('yr').drop("casual").drop("registered")

In [22]:
hour_df= df.select([ col(column).cast("double").alias(column) for column in df.columns])

In [23]:
hour_df

DataFrame[season: double, mnth: double, hr: double, holiday: double, weekday: double, workingday: double, weathersit: double, temp: double, atemp: double, hum: double, windspeed: double, cnt: double]

In [25]:
hour_df.show(5)

+------+----+---+-------+-------+----------+----------+----+------+----+---------+----+
|season|mnth| hr|holiday|weekday|workingday|weathersit|temp| atemp| hum|windspeed| cnt|
+------+----+---+-------+-------+----------+----------+----+------+----+---------+----+
|   1.0| 1.0|0.0|    0.0|    6.0|       0.0|       1.0|0.24|0.2879|0.81|      0.0|16.0|
|   1.0| 1.0|1.0|    0.0|    6.0|       0.0|       1.0|0.22|0.2727| 0.8|      0.0|40.0|
|   1.0| 1.0|2.0|    0.0|    6.0|       0.0|       1.0|0.22|0.2727| 0.8|      0.0|32.0|
|   1.0| 1.0|3.0|    0.0|    6.0|       0.0|       1.0|0.24|0.2879|0.75|      0.0|13.0|
|   1.0| 1.0|4.0|    0.0|    6.0|       0.0|       1.0|0.24|0.2879|0.75|      0.0| 1.0|
+------+----+---+-------+-------+----------+----------+----+------+----+---------+----+
only showing top 5 rows



In [26]:


train_df, test_df = hour_df.randomSplit([0.7, 0.3])
train_df.cache()
test_df.cache

<bound method DataFrame.cache of DataFrame[season: double, mnth: double, hr: double, holiday: double, weekday: double, workingday: double, weathersit: double, temp: double, atemp: double, hum: double, windspeed: double, cnt: double]>

In [28]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import  StringIndexer,  VectorIndexer,VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor

In [29]:
print("setup pipeline")

setup pipeline


In [34]:
featuresCols = hour_df.columns[:-1]
print(featuresCols) 

['season', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed']


In [56]:
vectorAssembler = VectorAssembler(inputCols=featuresCols, outputCol="aFeatures")
vectorIndexer = VectorIndexer(inputCol="aFeatures", outputCol="features", maxCategories=24)
dt = DecisionTreeRegressor(labelCol="cnt",featuresCol= 'features',maxDepth=10,impurity="variance",maxBins=100)
dt_pipeline = Pipeline(stages=[vectorAssembler,vectorIndexer ,dt])

In [57]:
dt_pipeline.getStages()

[VectorAssembler_41929154221977c48ed6,
 VectorIndexer_49399091be0dea07b623,
 DecisionTreeRegressor_4e3491e00fd98965b2b6]

In [58]:
print("train model")
dt_pipelineModel = dt_pipeline.fit(train_df)

train model


In [59]:
dt_pipelineModel.stages[2]

DecisionTreeRegressionModel (uid=DecisionTreeRegressor_4e3491e00fd98965b2b6) of depth 10 with 1791 nodes

In [60]:
print(dt_pipelineModel.stages[2].toDebugString[:980])

DecisionTreeRegressionModel (uid=DecisionTreeRegressor_4e3491e00fd98965b2b6) of depth 10 with 1791 nodes
  If (feature 2 in {0.0,1.0,2.0,3.0,4.0,5.0,6.0,22.0,23.0})
   If (feature 2 in {0.0,1.0,2.0,3.0,4.0,5.0})
    If (feature 2 in {2.0,3.0,4.0,5.0})
     If (feature 4 in {1.0,2.0,3.0,4.0,5.0})
      If (feature 2 in {2.0,3.0,4.0})
       If (feature 2 in {3.0,4.0})
        If (feature 1 in {0.0,1.0,2.0,3.0,11.0})
         If (feature 0 in {0.0})
          If (feature 7 <= 0.33)
           If (feature 1 in {1.0})
            Predict: 1.8611111111111112
           Else (feature 1 not in {1.0})
            Predict: 2.5789473684210527
          Else (feature 7 > 0.33)
           If (feature 2 in {4.0})
            Predict: 2.3333333333333335
           Else (feature 2 not in {4.0})
            Predict: 4.105263157894737
         Else (feature 0 not in {0.0})
          If (feature 7 <= 0.47)
           If (feature 0 in {1.0})
            Predict: 3.403225806451613
    


In [61]:
print("predict")
predicted = dt_pipelineModel.transform(test_df).select('season', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', \
                     'weathersit', 'temp', 'atemp', 'hum', 'windspeed','cnt','prediction').show(10)
print(predicted)

predict
+------+----+---+-------+-------+----------+----------+----+------+----+---------+----+------------------+
|season|mnth| hr|holiday|weekday|workingday|weathersit|temp| atemp| hum|windspeed| cnt|        prediction|
+------+----+---+-------+-------+----------+----------+----+------+----+---------+----+------------------+
|   1.0| 1.0|0.0|    0.0|    0.0|       0.0|       1.0|0.04|0.0758|0.57|   0.1045|22.0|              15.0|
|   1.0| 1.0|0.0|    0.0|    0.0|       0.0|       1.0|0.26| 0.303|0.56|      0.0|39.0|28.764705882352942|
|   1.0| 1.0|0.0|    0.0|    0.0|       0.0|       1.0|0.38|0.3939| 0.4|   0.2836|91.0|             56.25|
|   1.0| 1.0|0.0|    0.0|    1.0|       1.0|       1.0|0.12|0.1212| 0.5|   0.2836| 5.0|             15.44|
|   1.0| 1.0|0.0|    0.0|    1.0|       1.0|       1.0|0.22| 0.197|0.44|   0.3582| 5.0|             15.44|
|   1.0| 1.0|0.0|    0.0|    3.0|       1.0|       2.0|0.22|0.2273|0.69|   0.1343|17.0|10.090909090909092|
|   1.0| 1.0|0.0|    0.0|    

In [62]:
from pyspark.ml.evaluation import RegressionEvaluator

In [63]:
print("eval model")

eval model


In [64]:
evaluator = RegressionEvaluator(labelCol='cnt', predictionCol='prediction', metricName="rmse")

In [65]:
predictions = dt_pipelineModel.transform(test_df)
auc = evaluator.evaluate(predictions)
print(auc)

79.97113484730437
